In [1]:
import pandas as pd
import time
import os
import requests
from riotwatcher import LolWatcher, ApiError
from dotenv import load_dotenv

# ==========================================
# CONFIGURACIÓN
# ==========================================
load_dotenv()

# --- SI NO USAS .ENV, PON TU KEY DIRECTA AQUÍ ---
API_KEY = os.getenv("RIOT_API_KEY") 

RIOT_ID_NAME = "Stitch"
RIOT_ID_TAG = "FPMR"
REGION = "AMERICAS"      # Para buscar la cuenta (PUUID)
PLATFORM = "LA2"         # Para buscar las partidas (match_v5)
QUEUE_TYPE = 420         # 440 = Flex, 420 = SoloQ
CANTIDAD_OBJETIVO = 150  # Número de partidas a descargar
MINUTOS_DE_INTERES = [5, 10, 15] 

watcher = LolWatcher(API_KEY)

# ==========================================
# 1. OBTENCIÓN DE PUUID (ROBUSTO)
# ==========================================
def get_puuid(name, tag):
    print(f"--- Buscando usuario: {name} #{tag} ---")
    
    try:
        user = watcher.account.by_riot_id(REGION, name, tag)
        return user['puuid']
    except:
        pass 

    print("Usando método directo HTTP...")
    url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}"
    headers = {"X-Riot-Token": API_KEY}
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()['puuid']
        elif response.status_code == 403:
            raise ValueError("API KEY Vencida o Inválida.")
        else:
            print(f"Error API: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

# ==========================================
# 2. RECOPILACIÓN DE IDs
# ==========================================
def get_match_ids(puuid, count=200):
    all_matches = []
    start = 0
    
    print(f"--- Recopilando historial... ---")
    
    while len(all_matches) < count:
        try:
            remaining = count - len(all_matches)
            request_count = min(100, remaining)

            batch = watcher.match.matchlist_by_puuid(
                REGION, puuid, queue=QUEUE_TYPE, start=start, count=request_count
            )
            
            if not batch:
                print("Fin del historial disponible.")
                break
                
            all_matches.extend(batch)
            start += len(batch)
            
            print(f"IDs encontrados: {len(all_matches)} / {count}")
            time.sleep(0.8)
            
        except ApiError as err:
            print(f"Error obteniendo lista: {err}")
            break
            
    return all_matches

# ==========================================
# 3. EXTRACCIÓN DE DATOS (CON QUEUE_ID AGREGADO)
# ==========================================
def process_match(match_id, my_puuid):
    try:
        # A. Datos Generales
        match_detail = watcher.match.by_id(REGION, match_id)
        game_duration_min = match_detail['info']['gameDuration'] / 60
        
        # --- NUEVO: Extraemos el ID de la cola ---
        queue_id = match_detail['info']['queueId']
        
        if game_duration_min < 5: return None # Remake

        # B. Identificarme
        participants = match_detail['info']['participants']
        me = next((p for p in participants if p['puuid'] == my_puuid), None)
        
        if not me: return None
        
        my_id = me['participantId']
        my_team = me['teamId']
        my_role = me['teamPosition']
        
        if not my_role: return None

        # C. Identificar al RIVAL
        opponent = next(
            (p for p in participants if p['teamPosition'] == my_role and p['teamId'] != my_team),
            None
        )
        
        # --- EXTRACCIÓN DE VARIABLES ---
        
        row_data = {
            'match_id': match_id,
            'queue_id': queue_id,  # <--- AQUÍ GUARDAMOS SI ES FLEX (440) O SOLOQ (420)
            'win': 1 if me['win'] else 0,
            'champion': me['championName'],
            'role': my_role,
            'game_duration': round(game_duration_min, 2),
            
            # COMPONENTES PARA KDA
            'kills': me['kills'],
            'deaths': me['deaths'],
            'assists': me['assists'],
            
            # COMPONENTES PARA FARM
            'lane_minions': me['totalMinionsKilled'],     
            'jungle_minions': me['neutralMinionsKilled'], 
            
            # === VISIÓN ===
            'vision_score': me['visionScore'],                 
            'wards_placed': me['wardsPlaced'],                 
            'wards_killed': me['wardsKilled'],                 
            'control_wards_placed': me['detectorWardsPlaced']  
        }

        # D. Extraer Timeline (Minuto a Minuto)
        timeline = watcher.match.timeline_by_match(REGION, match_id)
        frames = timeline['info']['frames']

        for minute in MINUTOS_DE_INTERES:
            col_prefix = f"min{minute}"
            
            if minute >= len(frames):
                row_data[f'{col_prefix}_gold_diff'] = 0
                row_data[f'{col_prefix}_xp_diff'] = 0
                row_data[f'{col_prefix}_cs_diff'] = 0
                continue 

            frame_data = frames[minute]
            
            # Mis datos
            if str(my_id) not in frame_data['participantFrames']: continue
            
            my_frame = frame_data['participantFrames'][str(my_id)]
            my_gold = my_frame['totalGold']
            my_cs = my_frame['minionsKilled'] + my_frame['jungleMinionsKilled']
            
            row_data[f'{col_prefix}_gold'] = my_gold
            row_data[f'{col_prefix}_cs'] = my_cs

            # Rival
            if opponent:
                opp_id = opponent['participantId']
                if str(opp_id) in frame_data['participantFrames']:
                    opp_frame = frame_data['participantFrames'][str(opp_id)]
                    
                    opp_gold = opp_frame['totalGold']
                    opp_xp = opp_frame['xp']
                    opp_cs = opp_frame['minionsKilled'] + opp_frame['jungleMinionsKilled']
                    
                    my_xp = my_frame['xp']

                    row_data[f'{col_prefix}_gold_diff'] = my_gold - opp_gold
                    row_data[f'{col_prefix}_xp_diff'] = my_xp - opp_xp
                    row_data[f'{col_prefix}_cs_diff'] = my_cs - opp_cs
                else:
                    row_data[f'{col_prefix}_gold_diff'] = 0
                    row_data[f'{col_prefix}_xp_diff'] = 0
                    row_data[f'{col_prefix}_cs_diff'] = 0
            else:
                row_data[f'{col_prefix}_gold_diff'] = 0
                row_data[f'{col_prefix}_xp_diff'] = 0
                row_data[f'{col_prefix}_cs_diff'] = 0

        return row_data

    except Exception as e:
        return None

# ==========================================
# 4. EJECUCIÓN
# ==========================================
if __name__ == "__main__":
    puuid = get_puuid(RIOT_ID_NAME, RIOT_ID_TAG)

    if puuid:
        print(f"PUUID: {puuid}")
        match_ids = get_match_ids(puuid, count=CANTIDAD_OBJETIVO)
        
        if match_ids:
            print(f"\n--- Procesando {len(match_ids)} partidas... ---")
            dataset = []
            
            for i, mid in enumerate(match_ids):
                row = process_match(mid, puuid)
                if row:
                    dataset.append(row)
                
                if (i + 1) % 10 == 0:
                    print(f"Procesadas: {i + 1}/{len(match_ids)}")
                
                time.sleep(1.2) 

            if dataset:
                df = pd.DataFrame(dataset)
                filename = "Games_SoloQ.csv" # Mantenido original
                df.to_csv(filename, index=False)
                print(f"\n✅ ¡LISTO! Archivo guardado: {filename}")
                print(f"Total partidas: {len(df)}")
                print(f"Columnas: {list(df.columns)}")
            else:
                print("❌ No se pudieron extraer datos.")
        else:
            print("❌ No se encontraron partidas.")
    else:
        print("❌ Error obteniendo PUUID.")

--- Buscando usuario: Stitch #FPMR ---
Usando método directo HTTP...
PUUID: B05Qvn3L1ayxo9Sgn0qNW3s9ZLQ3X-N82E1OSUt0aWNfUbFzU_ojLBJ8ljxMxRIhirhO4x1aRPViTA
--- Recopilando historial... ---
IDs encontrados: 100 / 150
IDs encontrados: 117 / 150
Fin del historial disponible.

--- Procesando 117 partidas... ---
Procesadas: 10/117
Procesadas: 20/117
Procesadas: 30/117
Procesadas: 40/117
Procesadas: 50/117
Procesadas: 60/117
Procesadas: 70/117
Procesadas: 80/117
Procesadas: 90/117
Procesadas: 100/117
Procesadas: 110/117

✅ ¡LISTO! Archivo guardado: Games_SoloQ.csv
Total partidas: 114
Columnas: ['match_id', 'queue_id', 'win', 'champion', 'role', 'game_duration', 'kills', 'deaths', 'assists', 'lane_minions', 'jungle_minions', 'vision_score', 'wards_placed', 'wards_killed', 'control_wards_placed', 'min5_gold', 'min5_cs', 'min5_gold_diff', 'min5_xp_diff', 'min5_cs_diff', 'min10_gold', 'min10_cs', 'min10_gold_diff', 'min10_xp_diff', 'min10_cs_diff', 'min15_gold', 'min15_cs', 'min15_gold_diff', 'mi